# Imports

In [19]:
import torch
!pip install ogb
!pip install torch-geometric torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 KB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=e15c813b28e08bd3be76013c93008ca57904045529d8d2991f042b1e781f9435
  Stored in directory: /root/.cache/pip/wheels/04/bb/0d/2d02ec45f29c48d6192476bfb59c5a0e64b605e7212374dd15
Successfully built littleutils
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html


In [20]:
import multiprocessing as mp
import os
import pickle
import warnings
import torch
from functools import partial
from itertools import combinations
import numpy as np
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity as cos_sim
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import BallTree
from sklearn.preprocessing import StandardScaler, normalize
from ogb.nodeproppred import PygNodePropPredDataset
from tqdm import tqdm

from scipy.linalg import orthogonal_procrustes
from torch_geometric.datasets import Planetoid, Coauthor

In [3]:
# use google drive for saving and loading information
from google.colab import drive
import pickle
import os

drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/L45_project/'

Mounted at /content/drive


# Cora Data Set Stats

In [4]:
cora_dataset = Planetoid("/tmp/cora", name="cora", split="full")
cora_data = cora_dataset[0]

Processing...
Done!


In [21]:
cs_dataset = Coauthor("/tmp/coauthor", name="CS")
cs_data = cs_dataset[0]
cs_data

Processing...
Done!


Data(x=[18333, 6805], edge_index=[2, 163788], y=[18333])

In [22]:
d_name = "ogbn-arxiv"

dataset = PygNodePropPredDataset(name = d_name)

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
arxiv_data = dataset[0]
arxiv_data.y = arxiv_data.y.squeeze()
arxiv_data.node_year = arxiv_data.node_year.squeeze()
arxiv_data

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:10<00:00,  8.08it/s]


Extracting dataset/arxiv.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 2340.57it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 5047.30it/s]

Saving...



Done!


Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343], y=[169343])

In [ ]:
# Create a random list of indices with 10 items from each class for 2nd cossim
cora_labels = cora_dataset[0].y.detach().numpy()
cora_num_classes = 7
num_samples = 10
cora_indices = []
for i in range(cora_num_classes):
  class_i = np.random.choice(np.where(cora_labels == i)[0], size=num_samples, replace=False)
  print(class_i)
  cora_indices += class_i.tolist()


[2181 2180 2653   52 2470 1801   10 2333  274 1435]
[ 126  103 1180 1781 1337 1564  133  176  958 1991]
[1263 2492 1383 1738   58   90 2530 1492 1148 1759]
[1014 2568  812  309  821  827 2140 1065  512 2126]
[ 151 1774    1   43 2153 2694 2386   78 2679 2121]
[1960 2188 2184  925 2506 1338  113 1029  252  438]
[  31 2638 1900 2135 2161  123   26  850  537  331]


# Metric Calculations

In [15]:
# Adapted from https://github.com/SGDE2020/embedding_stability/blob/master/lib/tools/comparison.py

class Comparison:

    def __init__(self, emb_dir, embeddings, num_nodes, file_prefix):
        """
        emb_dir: str, path where embeddings are saved
        embeddings: list, list of strings that specify the embedding files
        """
        self.dir = emb_dir
        self.embeddings = embeddings
        self.pairs = self._combinations(embeddings)
        self.num_vertices = num_nodes

        # use file names without numbering to mark result files
        self.file_prefix = file_prefix

    def _combinations(self, emb_list):
        """ Computes all different comparison pairs of a list of embeddings """
        return [pair for pair in combinations(emb_list, 2)]

    def _analyse_jaccard(self, queries, nodes, k, pair):
        """This function is called in the multiprocessing of jaccard score"""
        indices_0 = np.asarray(queries[pair[0]])
        indices_1 = np.asarray(queries[pair[1]])
        nodes = np.asarray(nodes)
        jaccard_score = {}
        for i in range(len(nodes)):
            jaccard_score[nodes[i]] = \
                len(np.intersect1d(indices_0[i, 1:(k + 1)], indices_1[i, 1:(k + 1)], assume_unique=True)) / \
                len(np.union1d(indices_0[i, 1:(k + 1)], indices_1[i, 1:(k + 1)]))
        return list(jaccard_score.values())

    def _analyse_second_cossim(self, queries, normed_embs, nodes, k, pair):
        """
        This function is called in the multiprocessing of the second order cosine similarity.
        """

        # Convert the indices of nearest neighbors back into numpy
        indices_0 = np.asarray(queries[pair[0]])
        indices_1 = np.asarray(queries[pair[1]])

        # Convert the embeddings and nodes back into numpy
        norm_emb_0 = np.asarray(normed_embs[pair[0]])
        norm_emb_1 = np.asarray(normed_embs[pair[1]])
        nodes = np.asarray(nodes)

        # Compute the second order cosine similarity
        pair_results = []
        for i in range(len(nodes)):
            # Build the set of nearest neighbors w.r.t. both embeddings
            # Use indices from 1 to k+1, because the first entry will always be the node itself
            neighbors_union = np.union1d(indices_0[i, 1:(k + 1)], indices_1[i, 1:(k + 1)])

            # Vectors of cosine similarity values of nearest neighbors. There was an error in the original source code, did not use norm_emb_1 for m1
            m0 = cos_sim(norm_emb_0[neighbors_union], norm_emb_0[nodes[i]].reshape(1, -1))
            m1 = cos_sim(norm_emb_1[neighbors_union], norm_emb_1[nodes[i]].reshape(1, -1))

            # Flatten output matrix
            assert m0.shape[1] == 1 and m1.shape[1] == 1, "m0 and m1 should only have a single variable in the second dimension"
            m0 = m0.flatten()
            m1 = m1.flatten()

            # Cosine similarity between similarity vectors
            pair_results.append(float(1-cosine(m0,m1)))
        return pair_results

    def k_nearest_neighbors(self, nodes=None, append=False, samples=100, k=10, jaccard=False, load=False,
                            kload_size=100, save=True, save_path=None, num_processes=4):
        """
        Computes the k nearest neighbors to some specified nodes with respect to cosine similarity. As an intermediate
        step, it computes the 100 nearest neighbors and saves them to file. Based on these neighbors, the k-nn overlaps
        are computed.
        Args:
            nodes: list, that specifies the nodes
            append: bool, if nodes are specified, whether additional nodes will be sampled and added to nodes
            samples: int, number of nodes that will be sampled
            k: int, number of neighbors that will be used in the comparison
            jaccard: bool, whether jaccard score or overlap will be used as similarity measure
            load: bool, whether a file of computed neighbors will be used
            kload_size: Size of k in knn file to load
            save: bool, whether the results should be saved in a text file
            save_path: str, path where the file will be saved
            num_processes: number of random processes in parallelization
        Returns:
            dict, "nodes": array of nodes, "overlaps": array of overlaps, columns are values per node; or array of
            jaccard scores
        """
        # Handle the nodes input: Whether to sample nodes, use given nodes, or both.
        nodes = self._get_nodes(nodes, samples, append)

        # Load k nearest neighbors to save time? Else we will have to compute them first.
        if load:
            file_name = self.file_prefix + "_" + str(kload_size) + "nns.pickle"
            with open(os.path.join(save_path, file_name), "rb") as pickle_file:
                queries = pickle.load(pickle_file)
            assert list(queries.keys()) == self.embeddings, ("Keys of loaded queries do not match"
                                                             " with available embeddings")

        # Normalizing the embeddings to be able to use distance as a proxy for cosine similarity
        # BallTree from sklearn is used to compute the neighbors efficiently
        else:
            queries = self.nearest_neighbors_queries(nodes, k, save_path)
        # Store the naive overlaps for all pairs

        print(f"\n\n {queries.keys()} \n")

        # Use multiprocessing to speed up overlap computation.
        # Too much data is passed to the processes which makes it inefficient.
        # Possibly, it is faster to store the data as a file as an intermediate step.
        # only run if less than 100 neighbors are queried, as too large neighborhoods may cause memory issues when
        # distributing tasks
        if num_processes > 1 and k <= 100:
            with mp.Pool(num_processes) as p:
                # arguments passed in multiprocessing must be pickable
                p_queries = queries
                p_nodes = nodes.tolist()
                if jaccard:
                    multiprocess_func = partial(self._analyse_jaccard, p_queries, p_nodes, k)
                else:
                    multiprocess_func = partial(self._analyse_knn, p_queries, p_nodes, k)
                li_overlap = []
                for result in tqdm(p.imap(multiprocess_func, self.pairs), total=len(self.pairs)):
                  li_overlap.append(result)
        else:
            if jaccard:
                li_overlap = [self._analyse_jaccard(queries, nodes.tolist(), k, pair) for pair in self.pairs]
            else:
                li_overlap = [self._analyse_knn(queries, nodes.tolist(), k, pair) for pair in self.pairs]

        # Convert the result into numpy
        overlap = np.asarray(li_overlap)

        # Save the results
        if jaccard:
            nodes_suffix = "jaccard_nodes"
            scores_suffix = f"{k}nn_jaccard"
        else:
            nodes_suffix = "overlap_nodes"
            scores_suffix = f"{k}nn_overlap"
        if save is True:
            np.save(os.path.join(save_path, f"{self.file_prefix}_{nodes_suffix}"), nodes)
            np.save(os.path.join(save_path, f"{self.file_prefix}_{scores_suffix}"), overlap)
        return {"nodes": nodes, "overlaps": overlap}

    def jaccard_similarity(self, nodes=None, append=False, samples=100, k=10, load=False, kload_size=100, save=True,
                           save_path=None, num_processes=4):
        """
        Alias for k_nearest_neighbors with jaccard=True.
        See k_nearest_neighbors for detailed documentation.
        """
        return self.k_nearest_neighbors(nodes=nodes, append=append, samples=samples, k=k, jaccard=True, load=load,
                                        kload_size=kload_size, save=save, save_path=save_path,
                                        num_processes=num_processes)


    def second_order_cosine_similarity(self, nodes=None, append=False, num_samples=1000, k=10, load=True, save=False,
                                       save_path=None, num_processes=4):
        """ Computes second order cosine similarity.
        Args:
            nodes: list, that specifies the nodes
            append: bool, if nodes are specified, whether additional nodes will be sampled and added to nodes
            num_samples: int, number of nodes that will be sampled
            k: int, number of neighbors that will be used in the comparison
            load: bool, whether to load nearest neighbors from file
            save: bool, whether the results should be saved
            save_path: str, path where the file will be saved
            num_processes: number of random processes in parallelization
        Returns:
            nodes: numpy array of used nodes
            results: numpy array of similarity values of size (number of embedding pairs, number of nodes)
        """

        # Handle the nodes input: Whether to sample nodes, use given nodes, or both.
        nodes = self._get_nodes(nodes, num_samples, append)

        # Load required data: nearest neighbors, embeddings
        normed_embs = {}

        if load:
            # Load nearest neighbors from file
            file_name = self.file_prefix + "_" + str(k)+ "nns.pickle"
            with open(os.path.join(save_path, file_name), "rb") as pickle_file:
                queries = pickle.load(pickle_file)
            assert list(queries.keys()) == self.embeddings, ("Keys of loaded queries do not match with "
                                                             "available embeddings")
            for emb in tqdm(self.embeddings, desc="Loading nearest neighbor files"):
                normed_embs[emb] = normalize(self.read_embedding(os.path.join(self.dir, emb)), norm='l2', copy=False)
        else:
            queries, normed_embs = self.nearest_neighbors_queries(nodes, k, save_path, return_embeddings=True)

        # Start computation of second order cosine similarity
        # arguments passed in multiprocessing must be pickable
        p_normed_embs = dict([(key, norm_emb.tolist()) for key, norm_emb in normed_embs.items()])
        p_nodes = nodes.tolist()
        # Avoid multiprocessing on Colab, not sure if it works locally though
        if num_processes > 1 and k <= 100:
            with mp.Pool(num_processes) as p:
                li_results = p.map(partial(self._analyse_second_cossim, queries, p_normed_embs, p_nodes, k), self.pairs)
              # li_results = []
              # partial_func = partial(self._analyse_second_cossim, queries, p_normed_embs, p_nodes, k) 
              # for result in tqdm(p.imap(partial_func, self.pairs), total=len(self.pairs)):
              #   li_results.append(result)
        else:
          li_results = []
          for pair in tqdm(self.pairs, desc="Comparing embeddings"):
            li_results.append(self._analyse_second_cossim(queries, p_normed_embs, p_nodes, k, pair))

        results = np.asarray(li_results)

        if save is True:
            np.save(os.path.join(save_path, f"{self.file_prefix}_{k}nn_2nd_order_cossim"), results)

        return nodes, results

    def nearest_neighbors_queries(self, nodes, k, save_path, return_embeddings=False):
        """Uses a ball tree to compute the nearest neighbors in the embedding space"""
        queries = {}
        normed_embs = {}
        # Normalizing the embeddings to be able to use distance as a proxy for cosine similarity
        # BallTree from sklearn is used to compute the neighbors efficiently
        if return_embeddings:
            for emb in tqdm(self.embeddings, desc="Querying nearest neighbors"):
                normed_embs[emb] = normalize(self.read_embedding(os.path.join(self.dir, emb)), norm='l2', copy=False)
                ball_tree = BallTree(normed_embs[emb], leaf_size=40)
                queries[emb] = ball_tree.query(normed_embs[emb][nodes, :], k=k + 1, return_distance=False).tolist()
        else:
            for emb in tqdm(self.embeddings, desc="Querying nearest neighbors"):
                normalized_embedding = normalize(self.read_embedding(os.path.join(self.dir, emb)), norm='l2',
                                                 copy=False)
                ball_tree = BallTree(normalized_embedding, leaf_size=40)
                # Query the k+1 nearest neighbors, because a node will always be the closest neighbor to itself
                queries[emb] = ball_tree.query(normalized_embedding[nodes, :], k=k + 1, return_distance=False).tolist()

        # Save the computed neighbors to be able to skip the computation
        self.save_pickle(queries, save_path, self.file_prefix + "_" + str(k) + "nns")

        if return_embeddings:
            return queries, normed_embs
        else:
            return queries

    def sample_nodes(self, k):
        """
        Sample unique nodes of an embedding
        Args:
            k: int, number of nodes to sample
        Returns:
            numpy array of node ids of length k if k is smaller than the number of nodes available.
            Otherwise, all available nodes are returned.
        """
        vertices = np.arange(self.num_vertices)
        np.random.shuffle(vertices)
        return vertices[:min(k, self.num_vertices)]


    def cossim_analysis(self, save_path):
        """ Computes aligned cosine similarity values. Internally performs orthogonal transformation (Procrustes
        problem) between two embeddings and saves transformation matrices as well as vector of resulting errors
        """

        # Set up file naming
        results_suffix = "aligned_cossim"

        # Read the embeddings
        normed_embs = {}
        for emb in tqdm(self.embeddings, desc="Reading embeddings"):
            normed_embs[emb] = normalize(
                    self.read_embedding(os.path.join(self.dir, emb))[np.arange(self.num_vertices)], norm='l2',
                    copy=False)

        # Do the analysis
        emb_ind = -1
        results = []
        for pair in tqdm(self.pairs, desc="Comparing embeddings"):

            # only update first embedding if it does not change
            if emb_ind != pair[0]:
                emb_ind = pair[0]

            W1 = normed_embs[emb_ind]

            # transform W2 into W1 using procrustes matrix
            Q, _ = orthogonal_procrustes(normed_embs[pair[1]], normed_embs[pair[0]], check_finite=False)
            W2 = normed_embs[pair[1]].dot(Q)

            # Do 1-cosine to get the actual cosine similarity instead of cosine difference
            pair_results = np.array([1-cosine(W1[i], W2[i]) for i in range(self.num_vertices)])
            results.append(pair_results)

        results = np.asarray(results)
        np.save(os.path.join(save_path, f"{self.file_prefix}_{results_suffix}"), results)
        return np.arange(self.num_vertices), results

    def unaligned_cosine_analysis(self, save_path):
        """ Computes regular cosine similarity values, without alignment, between embedding pairs
        """

        # Set up file naming
        results_suffix = "cossim"

        # Read the embeddings
        normed_embs = {}
        for emb in tqdm(self.embeddings, desc="Reading embeddings"):
            # normed_embs[emb] = normalize(
            #         self.read_embedding(os.path.join(self.dir, emb))[np.arange(self.num_vertices)], norm='l2',
            #         copy=False)
            normed_embs[emb] = torch.nn.functional.normalize(
                    self.read_embedding(os.path.join(self.dir, emb))[np.arange(self.num_vertices)])

        # Do the analysis
        emb_ind = -1
        results = []
        cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
        for pair in tqdm(self.pairs, desc="Comparing embeddings"):

            # only update first embedding if it does not change
            if emb_ind != pair[0]:
                emb_ind = pair[0]

            # Get our two embedding matrics
            W1 = normed_embs[emb_ind]
            W2 = normed_embs[pair[1]]

            # # Do 1-cosine to get the actual cosine similarity instead of cosine distance
            # pair_results = np.array([1-cosine(W1[i], W2[i]) for i in range(self.num_vertices)])
            pair_results = cos(W1, W2)
            results.append(pair_results.cpu().detach().numpy())

        results = np.asarray(results)
        np.save(os.path.join(save_path, f"{self.file_prefix}_{results_suffix}"), results)
        return np.arange(self.num_vertices), results

    def pairwise_distance(self, nodes, save_path, norm=False):
        """Calculate the pairwise distance between all our embeddings. Save these to the given file path"""

        results_suffix = "euclidean_distance"

        # Read the embeddings
        normed_embs = {}
        for emb in tqdm(self.embeddings, desc="Reading embeddings"):
            # Option for calculating distance between normalized embeddings
            if norm:
              normed_embs[emb] = normalize(
                      self.read_embedding(os.path.join(self.dir, emb))[np.arange(self.num_vertices)], norm='l2',
                      copy=False)
            else:
              normed_embs[emb] = self.read_embedding(os.path.join(self.dir, emb))[np.arange(self.num_vertices)]

        # Do the analysis
        emb_ind = -1
        results = []
        for pair in tqdm(self.pairs, desc="Comparing embeddings"):

            # only update first embedding if it does not change
            if emb_ind != pair[0]:
                emb_ind = pair[0]

            # Get the distance between the two matrices of embeddings and add to results array
            # results.append(np.linalg.norm(normed_embs[emb_ind] - normed_embs[pair[1]], axis=1))
            results.append(torch.linalg.norm(normed_embs[emb_ind] - normed_embs[pair[1]], axis=1).cpu().detach().numpy())

        results = np.asarray(results)
        np.save(os.path.join(save_path, f"{self.file_prefix}_{results_suffix}"), results)


    def _get_nodes(self, nodes, num_samples, append):
        """
        Handles getting nodes for the experiments.
        Args:
            nodes: list, node ids
            num_samples: int, how many nodes should be sampled
            append: bool, whether to append sampled nodes to specified nodes
        Returns:
            numpy array of (sampled) node ids
        """
        if nodes is None:
            nodes = self.sample_nodes(num_samples)
        elif append is True:
            # allows specified nodes to be taken twice
            nodes.extend(self.sample_nodes(num_samples))
        return np.asarray(nodes)

    def save_pickle(self, obj, save_path, file_name):
        if save_path is None:
            save_path = os.getcwd()
        if file_name is None:
            # generate name of report from embedding input: use name of first embedding file without number information
            file_name = self.file_prefix
        with open(os.path.join(save_path, f"{file_name}.pickle"), "wb") as f:
            pickle.dump(obj, f)

    def get_combinations(self):
        return self.pairs

    def get_vertex_count(self):
        return self.num_vertices

    # The original code reorders the nodes by ID, but our nodes have the same ordering so there is no need to reorder
    def read_embedding(self, path):
      # node_embedding = torch.load(path, map_location=torch.device('cpu')).detach().numpy()
      node_embedding = torch.load(path)
      return node_embedding

In [5]:
# Code adapted from https://github.com/SGDE2020/embedding_stability/blob/master/similarity_tests/similarity_tests.py
TESTS= ["cossim", "jaccard", "2ndcos", "unalign_cossim", "dist"]

# Expects the the file names in the embedding directory to be of the form model-name_seed_emb.pt
def run_tests(embedding_dir, file_prefix, tests=TESTS, num_nodes=-1, knn_size=20, load_knn=False, kload_size=None,
                results_dir=None, num_processes=4, cossim_sec_indices=None):
    """
    Run specified similarity tests.
    Params:
        embedding_dir: str, path to directory where embeddings are stored
        file_prefix: the prefix for files for saving results
        tests: list of str, tests that will be conducted
        num_nodes: int, number of nodes of a graph that are used in the tests (just the number of nodes in the graph)
        knn_size: int, neighborhood size for knn test
        load_knn: bool, whether a stored knn matrix should be loaded
        nodeinfo_dir = str, path to directory where nodeinfo is stored (tables)
        results_dir = str, path to directory where results will be saved to
    """
    # Store results in the embedding directory in a results file if no other location is specified
    if results_dir is None:
      results_dir = embedding_dir + "results/"
    # Create results directory if it does not exist
    if not os.path.exists(results_dir):
        os.mkdir(results_dir) 

    # Computes a list of all _emb.pt files in a given directory. Assumes the directory being used only contains results from one architecture and data set
    fnames = sorted([f for f in os.listdir(embedding_dir) if f.endswith("_emb.pt")])
    print(fnames)

    if len(fnames) <= 1:
        print(f"Did not find any embeddings for in directory {embedding_dir}")
 
    else:
        # To be compatible with the original source code, we need a list of nodes indices we will be using in our metric computations
        # We use all the nodes
        nodes = [i for i in range(num_nodes)]

        # Allow us to use specific nodes for the second order cossine similarity as it can be very computational expensive
        cossim_nodes = nodes if cossim_sec_indices is None else cossim_sec_indices

        if load_knn and kload_size is None:
            kload_size = knn_size

        # Start tests
        comp = Comparison(emb_dir=embedding_dir, embeddings=fnames, num_nodes=num_nodes, file_prefix=file_prefix)
        if "cossim" in tests:
            print("Executing aligned cosine similarity")
            comp.cossim_analysis(save_path=results_dir)
        if "jaccard" in tests:
            print("Executing jaccard score")
            comp.jaccard_similarity(
                nodes=nodes, append=False, k=knn_size,
                load=load_knn, kload_size=kload_size, save=True, save_path=results_dir, num_processes=num_processes
            )
        if "2ndcos" in tests:
            print("Executing second order cosine similarity")
            comp.second_order_cosine_similarity(
                nodes=cossim_nodes, append=False, k=knn_size,
                save=True, save_path=results_dir, num_processes=num_processes, load=load_knn
            )
        if "unalign_cossim" in tests:
            print("Executing cosine similarity")
            comp.unaligned_cosine_analysis(save_path=results_dir)
        if "dist" in tests:
            print("Executing euclidean distance")
            comp.pairwise_distance(nodes=nodes, save_path=results_dir)



# Run metric calculations

In [28]:
# NUM_NODES_CORA = cora_data.x.shape[0]
# NUM_NODES_CORA = cs_data.x.shape[0]
# print(NUM_NODES_CORA)
NUM_NODES_CORA = 169343
# CHANGE these to match you directory naming structure
model_directory = "GraphSAGE-arvix/"
model_file_prefix = "GraphSAGE-arvix_analysis"
# model_directory = "GraphSAGE/"
# model_file_prefix = "GraphSAGE_analysis"

In [18]:
# Run the tests for tests=["cossim", "jaccard"], multiple processes work for these
# tests=["cossim", "jaccard", "unalign_cossim", "dist"]
run_tests(embedding_dir=file_path+model_directory, file_prefix=model_file_prefix, tests=["unalign_cossim", "dist"], num_nodes=NUM_NODES_CORA, num_processes=4)

['GATV2-arvix_1024258131_emb.pt', 'GATV2-arvix_116846604_emb.pt', 'GATV2-arvix_1221215631_emb.pt', 'GATV2-arvix_1517964140_emb.pt', 'GATV2-arvix_170173784_emb.pt', 'GATV2-arvix_1818027900_emb.pt', 'GATV2-arvix_1860537279_emb.pt', 'GATV2-arvix_1863727779_emb.pt', 'GATV2-arvix_2036056847_emb.pt', 'GATV2-arvix_2105922959_emb.pt', 'GATV2-arvix_2135956485_emb.pt', 'GATV2-arvix_2342954646_emb.pt', 'GATV2-arvix_2455059856_emb.pt', 'GATV2-arvix_2739899259_emb.pt', 'GATV2-arvix_2794978777_emb.pt', 'GATV2-arvix_2952735006_emb.pt', 'GATV2-arvix_3300171104_emb.pt', 'GATV2-arvix_3303475786_emb.pt', 'GATV2-arvix_361232447_emb.pt', 'GATV2-arvix_3647665043_emb.pt', 'GATV2-arvix_3692371949_emb.pt', 'GATV2-arvix_3710910636_emb.pt', 'GATV2-arvix_400225693_emb.pt', 'GATV2-arvix_4083009686_emb.pt', 'GATV2-arvix_4193977854_emb.pt', 'GATV2-arvix_516507873_emb.pt', 'GATV2-arvix_572297925_emb.pt', 'GATV2-arvix_806299656_emb.pt', 'GATV2-arvix_880019963_emb.pt', 'GATV2-arvix_89475662_emb.pt']
Executing cosine si

Comparing embeddings: 100%|██████████| 435/435 [00:02<00:00, 185.24it/s]


Executing euclidean distance


Comparing embeddings: 100%|██████████| 435/435 [00:00<00:00, 566.68it/s]


In [24]:
# Run the tests for tests=["2ndcos"], multiple processes does not work for this so only run on 1, but we can load the nearest neighbours from the previous metric calculations
run_tests(embedding_dir=file_path+model_directory, file_prefix=model_file_prefix, tests=["2ndcos"], num_nodes=NUM_NODES_CORA, num_processes=1, load_knn=True)

['FastRP-coauthor_1024258131_emb.pt', 'FastRP-coauthor_116846604_emb.pt', 'FastRP-coauthor_1221215631_emb.pt', 'FastRP-coauthor_1517964140_emb.pt', 'FastRP-coauthor_170173784_emb.pt', 'FastRP-coauthor_1818027900_emb.pt', 'FastRP-coauthor_1860537279_emb.pt', 'FastRP-coauthor_1863727779_emb.pt', 'FastRP-coauthor_2036056847_emb.pt', 'FastRP-coauthor_2105922959_emb.pt', 'FastRP-coauthor_2135956485_emb.pt', 'FastRP-coauthor_2342954646_emb.pt', 'FastRP-coauthor_2455059856_emb.pt', 'FastRP-coauthor_2739899259_emb.pt', 'FastRP-coauthor_2794978777_emb.pt', 'FastRP-coauthor_2952735006_emb.pt', 'FastRP-coauthor_3300171104_emb.pt', 'FastRP-coauthor_3303475786_emb.pt', 'FastRP-coauthor_361232447_emb.pt', 'FastRP-coauthor_3647665043_emb.pt', 'FastRP-coauthor_3692371949_emb.pt', 'FastRP-coauthor_3710910636_emb.pt', 'FastRP-coauthor_400225693_emb.pt', 'FastRP-coauthor_4083009686_emb.pt', 'FastRP-coauthor_4193977854_emb.pt', 'FastRP-coauthor_516507873_emb.pt', 'FastRP-coauthor_572297925_emb.pt', 'FastR

Comparing embeddings: 100%|██████████| 435/435 [1:57:56<00:00, 16.27s/it]


# Compute metric averages

In [24]:
def read_and_average_metric(file_suffix, class_labels, num_classes):
  results_arr = np.load(file_path+model_directory+"/results/"+model_file_prefix+"_"+file_suffix+".npy", mmap_mode='r')
  results_flat = results_arr.flatten() 
  # Compute the 5 number summary for the data across all classes, better than mean and std for potentially skewed distribution
  q1, q2, q3 = np.percentile(results_flat, [25,50,75])
  min, max = results_flat.min(), results_flat.max()
  # Print results
  print("TOTAL STATS")
  print("q1=" + str(q1) + ", q2=" + str(q2) + ", q3=" + str(q3) + ", min=" + str(min) + ", max=" + str(max))

  for i in range(num_classes):
    indices = np.where(class_labels==i)[0]
    # Filter the results by class
    class_vals = results_arr[:, indices]
    # Compute stats for the class
    q1_c, q2_c, q3_c = np.percentile(class_vals, [25,50,75])
    min_c, max_c = class_vals.min(), class_vals.max()
    # Print results
    print("CLASS STATS " + str(i))
    print("q1=" + str(q1_c), ", q2=" + str(q2_c) + ", q3=" + str(q3_c) + ", min=" + str(min_c) + ", max=" + str(max_c))


In [29]:
# Read and compute aligned cosine metric
read_and_average_metric("aligned_cossim", arxiv_data.y, 40)

TOTAL STATS
q1=0.9804521799087524, q2=0.9856035709381104, q3=0.9896413087844849, min=0.8768943548202515, max=0.9987865686416626
CLASS STATS 0
q1=0.9786377847194672 , q2=0.9836071133613586, q3=0.987501323223114, min=0.9066882729530334, max=0.997322678565979
CLASS STATS 1
q1=0.9822952151298523 , q2=0.9863936901092529, q3=0.9895402789115906, min=0.9296234846115112, max=0.9976093173027039
CLASS STATS 2
q1=0.9770395159721375 , q2=0.9825584292411804, q3=0.9868589639663696, min=0.8910394906997681, max=0.9969831705093384
CLASS STATS 3
q1=0.9759956002235413 , q2=0.9809478223323822, q3=0.984931543469429, min=0.9006055593490601, max=0.996711015701294
CLASS STATS 4
q1=0.9794051647186279 , q2=0.984146386384964, q3=0.9877861738204956, min=0.9109001755714417, max=0.9976317882537842
CLASS STATS 5
q1=0.9775042533874512 , q2=0.9826571643352509, q3=0.986751601099968, min=0.9008263349533081, max=0.9974732398986816
CLASS STATS 6
q1=0.9770769625902176 , q2=0.9820587635040283, q3=0.9860889762639999, min=0.89

In [ ]:
# Read and compute jaccard metric
read_and_average_metric("20nn_jaccard", cs_data.y, 15)

In [ ]:
# Read and compute 2nd order cosine similarity metric
read_and_average_metric("20nn_2nd_order_cossim", cs_data.y, 15)

In [30]:
# Read and compute 2nd order cosine similarity metric
read_and_average_metric("cossim", arxiv_data.y, 40)

TOTAL STATS
q1=-0.05706239491701126, q2=0.0012373477220535278, q3=0.05968799442052841, min=-0.60218567, max=0.5487026
CLASS STATS 0
q1=-0.05535129830241203 , q2=0.001974891871213913, q3=0.0600893497467041, min=-0.5493524, max=0.4590387
CLASS STATS 1
q1=-0.05713629722595215 , q2=0.002160934265702963, q3=0.0612361803650856, min=-0.40131307, max=0.3852083
CLASS STATS 2
q1=-0.0579276978969574 , q2=0.0005013570189476013, q3=0.0587909035384655, min=-0.5507677, max=0.53250194
CLASS STATS 3
q1=-0.05829516984522343 , q2=0.0008924249559640884, q3=0.06012171879410744, min=-0.44720912, max=0.4109311
CLASS STATS 4
q1=-0.05579972080886364 , q2=0.0022803982719779015, q3=0.060563549399375916, min=-0.4598479, max=0.46122307
CLASS STATS 5
q1=-0.055147355422377586 , q2=0.00334305502474308, q3=0.061872776597738266, min=-0.5081487, max=0.4792943
CLASS STATS 6
q1=-0.05843238905072212 , q2=0.0005317078903317451, q3=0.0596251655369997, min=-0.4485746, max=0.420034
CLASS STATS 7
q1=-0.057634538039565086 , q2=0

In [32]:
read_and_average_metric("euclidean_distance", arxiv_data.y, 40)

TOTAL STATS
q1=7.274916172027588, q2=9.90475845336914, q3=12.349464416503906, min=2.6417778, max=46.612045
CLASS STATS 0
q1=7.110958576202393 , q2=9.589373588562012, q3=11.73887586593628, min=3.091793, max=22.681246
CLASS STATS 1
q1=7.228184700012207 , q2=10.069816589355469, q3=12.550368309020996, min=3.6638339, max=29.69075
CLASS STATS 2
q1=7.528102397918701 , q2=10.139183044433594, q3=12.537871360778809, min=3.08366, max=31.26897
CLASS STATS 3
q1=6.139352917671204 , q2=8.350060939788818, q3=10.30005431175232, min=3.0168195, max=29.157015
CLASS STATS 4
q1=7.052367806434631 , q2=9.536607265472412, q3=11.87348222732544, min=2.949913, max=42.66458
CLASS STATS 5
q1=6.458626747131348 , q2=9.0177903175354, q3=11.195642948150635, min=2.721705, max=26.532766
CLASS STATS 6
q1=6.1413127183914185 , q2=8.454389095306396, q3=10.767720699310303, min=2.8874793, max=30.27789
CLASS STATS 7
q1=6.307627201080322 , q2=8.6769380569458, q3=10.621937274932861, min=2.8875403, max=33.970623
CLASS STATS 8
q1=7

To do:
- Store the nodes being used if we're only using a subset
- Subsets for second_order_cosine_similarity for larger dataset